In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import scipy.misc
import pickle

%matplotlib inline

In [2]:
tf.enable_eager_execution()

In [ ]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
def resize_datasets(X):
    X_new = []
    for i in range(X.shape[0]):
        if(i % 100 == 0):
            print(i)
        img = scipy.ndimage.interpolation.zoom(X[i], (8.0, 8.0, 1.0))
        X_new.append(img)
    X_new = np.asarray(X_new, dtype=np.float32)
    return X_new.reshape([X.shape[0], 256, 256, 3])

In [ ]:
print('Resizing Validation...')
X_val = resize_datasets(X_val)

In [ ]:
print('Resizing Test...')
X_test = resize_datasets(X_test)

In [ ]:
print('Resizing Training...')
X_train = resize_datasets(X_train)

In [ ]:
X_train.shape

In [ ]:
pickle_out = open("train_dataset.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("validation_dataset.pickle","rb")
X_val = pickle.load(pickle_in)

In [28]:
import pickle
pickle_in = open("dataset.pickle","rb")
X_train = pickle.load(pickle_in)
y_train = pickle.load(pickle_in)
X_test = pickle.load(pickle_in)
y_test = pickle.load(pickle_in)
X_val = pickle.load(pickle_in)
y_val = pickle.load(pickle_in)

In [29]:
X_test[:30] = X_train[:30]
y_test[:30] = y_train[:30]

In [30]:
training_set_size = 200
test_set_size = 100
valid_set_size = 50

In [31]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

In [32]:
def flatten(x):
    """    
    Input:
    - TensorFlow Tensor of shape (N, D1, ..., DM)
    
    Output:
    - TensorFlow Tensor of shape (N, D1 * ... * DM)
    """
    N = tf.shape(x)[0]
    return tf.reshape(x, (N, -1))

In [ ]:
inp = tf.get_variable(name = 'input', shape=[1,256,256,3], dtype=tf.float32)

In [33]:
#globals used in the assignment
IMG_SHAPE = [256, 256, 3] # height, width, channels
NUM_CLASSES = 10

# training hyperparameters
LEARNING_RATE = 1e-4
MOMENTUM = 0.9
BATCH_SIZE = 16
EPOCHS = 30
DISPLAY_STEP = 1
VALIDATION_STEP = 1000
SAVE_STEP = 1000
CKPT_PATH = './ckpt'
SUMMARY_PATH = './summary'

# net architecture hyperparamaters
LAMBDA = 5e-4 #for weight decay
DROPOUT = 0.5

# test hyper parameters
K_PATCHES = 5
TOP_K = 5

In [34]:
class AlexNet(tfe.Network):
    def __init__(self, training):
        super(AlexNet, self).__init__()
        self.training = training
        
        # convolutional layers
        conv_init = tf.contrib.layers.xavier_initializer_conv2d()
        
        self.conv1 = self.track_layer(
            tf.layers.Conv2D(
                96,
                11,
                4,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool1 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv2 = self.track_layer(
            tf.layers.Conv2D(
                256,
                5,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool2 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv3 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv4 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv5 = self.track_layer(
            tf.layers.Conv2D(
                256,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool5 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        # fully connected layers

        fc_init = tf.contrib.layers.xavier_initializer()

        self.fc1 = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        self.drop1 = self.track_layer(tf.layers.Dropout(DROPOUT))
        
        #attention estimators
        self.atten1_fc = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        
        self.atten2_fc = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        
        self.out = self.track_layer(
            tf.layers.Dense(NUM_CLASSES, kernel_initializer=fc_init))

    def call(self, x):
        """ Function that executes the model """
        output = self.conv1(x)
        #print("After conv1 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        #print("After normalisation1 : " + str(output.numpy().shape))
        output = self.pool1(output)
        #print("After pool1 : " + str(output.numpy().shape))

        output = self.conv2(output)
        #print("After conv2 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        #print("After normalisation2 : " + str(output.numpy().shape))
        output = self.pool2(output)
        #print("After pool2 : " + str(output.numpy().shape))

        output = self.conv3(output)
        #print("After conv3 : " + str(output.numpy().shape))

        output = self.conv4(output)
        #print("After conv4 : " + str(output.numpy().shape))
        conv4_output = output

        output = self.conv5(output)
        #print("After conv5 : " + str(output.numpy().shape))
        conv5_output = output
        
        output = self.pool5(output)
        #print("After pool5 : " + str(output.numpy().shape))

        output = tf.layers.flatten(output)
        #print("After flattening : " + str(output.numpy().shape))

        output = self.fc1(output)
        #print("After fc1 : " + str(output.numpy().shape))
        if self.training:
            output = self.drop1(output)
            #print("After drop1 : " + str(output.numpy().shape))
        
        #logic for attention estimators
        g = tf.reshape(output, (output.shape[0], output.shape[1], 1))
        #print("G dimensions : " + str(g.numpy().shape))
        
#         print("\nATTENTION 1 :-")
        atten1_flattened_output = tf.reshape(conv4_output, (conv4_output.shape[0], conv4_output.shape[1] * conv4_output.shape[2], conv4_output.shape[3]))
#         print("After Flattening : " + str(atten1_flattened_output.numpy().shape))
        atten1_output = self.atten1_fc(atten1_flattened_output)
#         print("After FC : " + str(atten1_output.numpy().shape))
        atten1_output = tf.matmul(atten1_output, g)
#         print("After Dot Product with G : " + str(atten1_output.numpy().shape))
        atten1_output = tf.reshape(atten1_output, (atten1_output.shape[0], atten1_output.shape[2], atten1_output.shape[1]))
        attention1_values = atten1_output
        atten1_output = tf.nn.softmax(atten1_output)
#         print("After Reshaping and applying Softmax : " + str(atten1_output.numpy().shape))
        atten1_output = tf.matmul(atten1_output, atten1_flattened_output)
#         print("G_a1 Dimensions : " + str(atten1_output.numpy().shape))
        
        #print("\nATTENTION 2 :-")
        atten2_flattened_output = tf.reshape(conv5_output, (conv5_output.shape[0], conv5_output.shape[1] * conv5_output.shape[2], conv5_output.shape[3]))
        #print("After Flattening : " + str(atten2_flattened_output.numpy().shape))
        atten2_output = self.atten2_fc(atten2_flattened_output)
        #print("After FC : " + str(atten2_output.numpy().shape))
        atten2_output = tf.matmul(atten2_output, g)
        #print("After Dot Product with G : " + str(atten2_output.numpy().shape))
        atten2_output = tf.reshape(atten2_output, (atten2_output.shape[0], atten2_output.shape[2], atten2_output.shape[1]))
        attention2_values = atten2_output
        atten2_output = tf.nn.softmax(atten2_output)
        #print("After Reshaping and applying Softmax : " + str(atten2_output.numpy().shape))
        atten2_output = tf.matmul(atten2_output, atten2_flattened_output)
        #print("G_a1 Dimensions : " + str(atten2_output.numpy().shape))
        
        #print("\nCOMBINING BY OPTION 1 :-")
        output = tf.concat([atten1_output, atten2_output], 2)
        output = tf.reshape(output, (output.shape[0], output.shape[2]))
        #print("Option 1 Dimensions : " + str(output.numpy().shape))
        output = self.out(output)

        return output,attention1_values,attention2_values

In [35]:
def loss(net, mode, x, y):
    """
    Computes the loss for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the loss between the predictions on the images and the groundtruths
    """
    pred = net(x)[0]
    y = tf.one_hot(y, NUM_CLASSES)

    loss_value = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=pred)
    weight_decay = tf.reduce_sum(LAMBDA * tf.stack([tf.nn.l2_loss(v) for v in net.variables]))

    total_loss = loss_value + weight_decay

    tf.contrib.summary.scalar(mode + '/loss', total_loss)

    return total_loss

In [36]:
def accuracy(net, mode, x, y):
    """
    Computes the accuracy for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the accuracy of the predictions on the images and the groundtruths
    """
    pred = tf.nn.softmax(net(x)[0])
    y = tf.one_hot(y, NUM_CLASSES)

    accuracy_value = tf.reduce_sum(
                tf.cast(
                    tf.equal(
                        tf.argmax(pred, axis=1, output_type=tf.int64),
                        tf.argmax(y, axis=1, output_type=tf.int64)
                    ),
                    dtype=tf.float32
                ) 
            ) / float(pred.shape[0].value)

    tf.contrib.summary.scalar(mode +'/accuracy', accuracy_value)

    return accuracy_value

In [37]:
train_dset = Dataset(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=valid_set_size, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=test_set_size)

In [38]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
#     print(y)
    if t > 5: break

0 (16, 256, 256, 3) (16,)
1 (16, 256, 256, 3) (16,)
2 (16, 256, 256, 3) (16,)
3 (16, 256, 256, 3) (16,)
4 (16, 256, 256, 3) (16,)
5 (16, 256, 256, 3) (16,)
6 (16, 256, 256, 3) (16,)


In [39]:
#used for printing the log with the timestamp
logging = tf.logging
logging.set_verbosity(logging.INFO)
def log_msg(msg):
       logging.info(f'{time.ctime()}: {msg}') 

In [40]:
DISPLAY_STEP = 1
VALIDATION_STEP = 5
SAVE_STEP = 10

In [41]:
import time
def train(net):
    """
    Training procedure
    """
    start_time = time.time()
    step_time = 0.0
    
    writer = tf.contrib.summary.create_summary_file_writer(SUMMARY_PATH)
#     optimizer = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE, momentum=MOMENTUM)
    optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
    global_step = tf.train.get_or_create_global_step()
    epoch = tfe.Variable(0, name='epoch', dtype=tf.float32, trainable=False)
    all_variables = (net.variables + optimizer.variables() + [global_step] + [epoch])
    
    with writer.as_default():
        with tf.contrib.summary.record_summaries_every_n_global_steps(DISPLAY_STEP):
            for e in range(int(epoch.numpy()), EPOCHS):
                tf.assign(epoch, e)
                for (batch_i, (images, labels)) in enumerate(train_dset):
                    global_step = tf.train.get_global_step()
                    step = global_step.numpy() + 1
                    images = tf.to_float(images)
                    step_start_time = int(round(time.time() * 1000))
                    
                    optimizer.minimize(lambda: loss(net, 'train', images, labels), global_step=global_step)

                    step_end_time = int(round(time.time() * 1000))
                    step_time += step_end_time - step_start_time

                    if (step % DISPLAY_STEP) == 0:
                        l = loss(net, 'train', images, labels)
                        a = accuracy(net, 'train', images, labels).numpy()
                        log_msg('Epoch: {:03d} Step/Batch: {:09d} Step mean time: {:04d}ms \nLoss: {:.7f} Training accuracy: {:.4f}'.format(e, step, int(step_time / step), l, a))

                    if (step % VALIDATION_STEP) == 0:
                        for (_, (val_images, val_labels)) in enumerate(val_dset):
#                             val_images, val_labels = tfe.Iterator(val_dset).next()
                            l = loss(net, 'val', val_images, val_labels)
                            a = accuracy(net, 'val', val_images, val_labels).numpy()
                            int_time = time.time() - start_time
                            log_msg('Elapsed time: {:04d}ms --- Loss: {:.7f} Validation accuracy: {:.4f}'.format(int(int_time), l, a))

                    if (step % SAVE_STEP) == 0:
                        tfe.Saver(net.variables).save(os.path.join(CKPT_PATH, 'net.ckpt'), global_step=global_step)
                        log_msg('Variables saved')

In [42]:
alexnet = AlexNet(training=True)


Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but note that `track_layer` is no longer necessary or supported. Instead, `Layer` instances are tracked on attribute assignment (see the section of `tf.keras.Model`'s documentation on subclassing). Since the output of `track_layer` is often assigned to an attribute anyway, most code can be ported by simply removing the `track_layer` calls.

`tf.keras.Model` works with all TensorFlow `Layer` instances, including those from `tf.layers`, but switching to the `tf.keras.layers` versions along with the migration to `tf.keras.Model` is recommended, since it will preserve variable names. Feel free to import it with an alias to avoid excess typing :).


In [ ]:
if not os.path.exists(CKPT_PATH):
    os.makedirs(CKPT_PATH)
train(alexnet)

INFO:tensorflow:Sat Nov 17 20:53:38 2018: Epoch: 000 Step/Batch: 000000262 Step mean time: 0017ms 
Loss: 2.8710556 Training accuracy: 0.1875
INFO:tensorflow:Sat Nov 17 20:53:45 2018: Epoch: 000 Step/Batch: 000000263 Step mean time: 0032ms 
Loss: 3.0132735 Training accuracy: 0.1875
INFO:tensorflow:Sat Nov 17 20:53:52 2018: Epoch: 000 Step/Batch: 000000264 Step mean time: 0049ms 
Loss: 3.0348649 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 20:53:58 2018: Epoch: 000 Step/Batch: 000000265 Step mean time: 0064ms 
Loss: 3.0505106 Training accuracy: 0.0000
INFO:tensorflow:Sat Nov 17 20:54:06 2018: Elapsed time: 0034ms --- Loss: 3.0264702 Validation accuracy: 0.1000
INFO:tensorflow:Sat Nov 17 20:54:13 2018: Epoch: 000 Step/Batch: 000000266 Step mean time: 0080ms 
Loss: 3.0174997 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 20:54:19 2018: Epoch: 000 Step/Batch: 000000267 Step mean time: 0095ms 
Loss: 3.0163426 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 20:54:26 2018: E

INFO:tensorflow:Sat Nov 17 21:01:59 2018: Elapsed time: 0507ms --- Loss: 2.8308661 Validation accuracy: 0.1600
INFO:tensorflow:Sat Nov 17 21:02:00 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:02:10 2018: Epoch: 003 Step/Batch: 000000311 Step mean time: 0821ms 
Loss: 2.8231897 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:02:20 2018: Epoch: 003 Step/Batch: 000000312 Step mean time: 0837ms 
Loss: 2.8047483 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 21:02:24 2018: Epoch: 003 Step/Batch: 000000313 Step mean time: 0843ms 
Loss: 2.6855578 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:02:33 2018: Epoch: 004 Step/Batch: 000000314 Step mean time: 0858ms 
Loss: 2.7170143 Training accuracy: 0.1875
INFO:tensorflow:Sat Nov 17 21:02:42 2018: Epoch: 004 Step/Batch: 000000315 Step mean time: 0873ms 
Loss: 2.7710304 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 21:02:55 2018: Elapsed time: 0564ms --- Loss: 2.7848897 Validation accuracy: 0.2200
INFO:tensorflow

INFO:tensorflow:Sat Nov 17 21:11:36 2018: Epoch: 007 Step/Batch: 000000360 Step mean time: 1487ms 
Loss: 2.6306515 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:11:48 2018: Elapsed time: 1097ms --- Loss: 2.6729255 Validation accuracy: 0.1800
INFO:tensorflow:Sat Nov 17 21:11:49 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:11:59 2018: Epoch: 007 Step/Batch: 000000361 Step mean time: 1498ms 
Loss: 2.6812701 Training accuracy: 0.1875
INFO:tensorflow:Sat Nov 17 21:12:09 2018: Epoch: 007 Step/Batch: 000000362 Step mean time: 1511ms 
Loss: 2.6826246 Training accuracy: 0.1875
INFO:tensorflow:Sat Nov 17 21:12:19 2018: Epoch: 007 Step/Batch: 000000363 Step mean time: 1524ms 
Loss: 2.5294156 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:12:29 2018: Epoch: 007 Step/Batch: 000000364 Step mean time: 1536ms 
Loss: 2.4809437 Training accuracy: 0.3125
INFO:tensorflow:Sat Nov 17 21:12:34 2018: Epoch: 007 Step/Batch: 000000365 Step mean time: 1541ms 
Loss: 2.1858656 Training ac

INFO:tensorflow:Sat Nov 17 21:21:43 2018: Epoch: 011 Step/Batch: 000000409 Step mean time: 2023ms 
Loss: 2.2248774 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 21:21:53 2018: Epoch: 011 Step/Batch: 000000410 Step mean time: 2034ms 
Loss: 2.2881181 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 21:22:06 2018: Elapsed time: 1714ms --- Loss: 2.6168349 Validation accuracy: 0.1600
INFO:tensorflow:Sat Nov 17 21:22:07 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:22:17 2018: Epoch: 011 Step/Batch: 000000411 Step mean time: 2044ms 
Loss: 2.2404289 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:22:27 2018: Epoch: 011 Step/Batch: 000000412 Step mean time: 2054ms 
Loss: 1.9775455 Training accuracy: 0.5625
INFO:tensorflow:Sat Nov 17 21:22:37 2018: Epoch: 011 Step/Batch: 000000413 Step mean time: 2065ms 
Loss: 2.2727094 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:22:47 2018: Epoch: 011 Step/Batch: 000000414 Step mean time: 2075ms 
Loss: 2.1147606 Training ac

INFO:tensorflow:Sat Nov 17 21:31:50 2018: Epoch: 015 Step/Batch: 000000458 Step mean time: 2455ms 
Loss: 2.3939660 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:32:00 2018: Epoch: 015 Step/Batch: 000000459 Step mean time: 2463ms 
Loss: 2.7778006 Training accuracy: 0.3125
INFO:tensorflow:Sat Nov 17 21:32:10 2018: Epoch: 015 Step/Batch: 000000460 Step mean time: 2471ms 
Loss: 2.2262745 Training accuracy: 0.5000
INFO:tensorflow:Sat Nov 17 21:32:23 2018: Elapsed time: 2331ms --- Loss: 2.4606037 Validation accuracy: 0.2200
INFO:tensorflow:Sat Nov 17 21:32:23 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:32:33 2018: Epoch: 015 Step/Batch: 000000461 Step mean time: 2479ms 
Loss: 2.5257080 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:32:43 2018: Epoch: 015 Step/Batch: 000000462 Step mean time: 2487ms 
Loss: 2.3301249 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:32:54 2018: Epoch: 015 Step/Batch: 000000463 Step mean time: 2496ms 
Loss: 2.1908667 Training ac

In [ ]:
for (_, (test_images, test_labels)) in enumerate(test_dset):
    l = loss(alexnet, 'val', test_images, test_labels)
    a = accuracy(alexnet, 'val', test_images, test_labels).numpy()
log_msg('Final Test Loss: {:.7f} Test accuracy: {:.4f}'.format(l, a))